# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [47]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [48]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kulachi,31.9286,70.4592,103.21,27,6,20.85,PK,1692033918
1,1,puerto natales,-51.7236,-72.4875,36.34,93,98,4.21,CL,1692033918
2,2,kharan,28.5833,65.4167,92.39,9,51,6.13,PK,1692033918
3,3,puerto ayora,-0.7393,-90.3518,80.56,87,79,7.00,EC,1692033918
4,4,brookings,44.3114,-96.7984,70.05,69,20,18.41,US,1692033639


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
%%capture --no-display

# Configure the map plot

map = city_data_df.hvplot.points(
    x= 'Lng',
    y= 'Lat',
    size= 'Humidity',
    hover_cols= ["City"],
    colormap= 'viridis',
    tiles= 'OSM',
    geo= True,
    title= "City Humidity"
 
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [50]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df= city_data_df.loc[
    (city_data_df['Humidity']<= 80) &
    (city_data_df['Max Temp']>= 60) &
    (city_data_df['Max Temp']<= 100) &
    (city_data_df['Cloudiness']<= 20)
]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,brookings,44.3114,-96.7984,70.05,69,20,18.41,US,1692033639
12,12,edd,13.9297,41.6925,93.27,57,20,6.08,ER,1692033920
22,22,praia da vitoria,38.7333,-27.0667,80.76,73,20,4.38,PT,1692033860
23,23,sinop,41.7711,34.8709,73.65,72,0,5.23,TR,1692033922
28,28,bethel,41.3712,-73.4140,83.35,67,0,4.61,US,1692033923
31,31,port mathurin,-19.6833,63.4167,72.91,74,2,24.27,MU,1692033924
32,32,woodward,36.4337,-99.3904,78.03,51,0,11.50,US,1692033924
45,45,whitefish,48.4111,-114.3376,80.20,40,0,4.61,US,1692033928
50,50,capayan,-28.7610,-66.0502,85.80,15,4,4.92,AR,1692033929
52,52,san sebastian de la gomera,28.0916,-17.1133,89.38,54,0,4.61,ES,1692033929


### Step 3: Create a new DataFrame called `hotel_df`.

In [51]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= ideal_weather_df.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= " "

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
4,brookings,US,44.3114,-96.7984,69,
12,edd,ER,13.9297,41.6925,57,
22,praia da vitoria,PT,38.7333,-27.0667,73,
23,sinop,TR,41.7711,34.8709,72,
28,bethel,US,41.3712,-73.4140,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [52]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit": 20,
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= row["Lat"]
    longitude= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
brookings - nearest hotel: Quality Inn
edd - nearest hotel: No hotel found
praia da vitoria - nearest hotel: Salles
sinop - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
port mathurin - nearest hotel: Escale Vacances
woodward - nearest hotel: Sands Inn
whitefish - nearest hotel: The Firebrand Hotel
capayan - nearest hotel: Hosteria Concepción
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
rio grande - nearest hotel: Hotel Vila Moura Executivo
bang bua thong - nearest hotel: You and me
grand baie - nearest hotel: Safari
jamestown - nearest hotel: DoubleTree Jamestown
constantia - nearest hotel: Hotel Chérica
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
al kharijah - nearest hotel: Kharga Hotel
rabo de peixe - nearest hotel: Pico do Refugio
ciudad insurgentes - nearest hotel: Hotel Restaurant Californias
kailua-kona - nearest hotel: Kona Seaside Hotel
zhangjiakou shi xuanhua qu - nearest hotel: Century d

,City,Country,Lat,Lng,Humidity,Hotel Name
4,brookings,US,44.3114,-96.7984,69,Quality Inn
12,edd,ER,13.9297,41.6925,57,No hotel found
22,praia da vitoria,PT,38.7333,-27.0667,73,Salles
23,sinop,TR,41.7711,34.8709,72,No hotel found
28,bethel,US,41.3712,-73.4140,67,Hampton Inn Danbury
...,...,...,...,...,...,...
553,oxie,SE,55.5333,13.1000,65,Best Western Hotel Jägersro
557,tukrah,LY,32.5341,20.5791,61,No hotel found
558,birjand,IR,32.8663,59.2211,13,مهمانسرای جهانگردی بیرجند
569,yreka,US,41.7354,-122.6345,27,Best Western Minner's Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x= 'Lng',
    y= 'Lat',
    size= 'Humidity',
    colormap= 'viridis',
    tiles= 'OSM',
    geo= True,
    title= "Hotels Located in Cities with Ideal Weather Conditions",
    hover_cols= ["Hotel Name","Country"]
 
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)